# Imports and variables

In [16]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
import zipfile
import csv
import pandas as pd
import itertools
import re
from datetime import datetime
from utils.csv_headers import *
from collections import OrderedDict

In [18]:
CURRENT_PATH = os.getcwd()
UNZIPPED_DIR = os.path.join(CURRENT_PATH,'../content/downloads/unzipped')
OUTPUT_FILE = os.path.join(CURRENT_PATH,'../content/output_sql.tsv')
ENHANCED_FILE = os.path.join(CURRENT_PATH,'../content/enhanced-output_sql.tsv')
TABLE_DIR = os.path.join(CURRENT_PATH,'../content/tables/')

ENCODING = 'utf-8'
NEWLINE = ''
DELIMITER = '\t'

# Data preprocessing

## Unzip CSV project files

## CSV files merge

In [19]:
def is_integer(string):
    try:
        int(string)
        return True
    except ValueError as a:
        return False
    
def is_corrupted(dict_row):
    empty_column = None in dict_row
    is_application_id_not_integer = not is_integer(dict_row[APPLICATION_ID])   
    return empty_column or is_application_id_not_integer

In [21]:
with open(OUTPUT_FILE, 'w', encoding=ENCODING, newline=NEWLINE) as output_file:
    writer = csv.DictWriter(output_file, fieldnames=ORDERED_HEADERS, delimiter=DELIMITER)
    writer.writeheader()

    csv_files = sorted(os.listdir(UNZIPPED_DIR))

    for csv_file_name in csv_files:
        print(csv_file_name)

        with open(UNZIPPED_DIR + '/' + csv_file_name, 'r', encoding='ISO-8859-1', newline=NEWLINE) as csv_file_descriptor:
            reader = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')
            
            for index, dict_row in enumerate(reader):
                if is_corrupted(dict_row):
                    print(csv_file_name, index, dict_row[APPLICATION_ID])
                else:
                    upper_dict = dict()
                    for key, value in dict_row.items():
                        try:
                            # letter case uniformization
                            upper_dict[key.upper()] = value.replace("\n", "").replace("\t", " ")
                        except Exception as e:
                            print(e)
                            print(dict_row)
                            print(csv_file_name)
                            #raise e
                    writer.writerow(upper_dict)
                
        break

RePORTER_PRJ_C_FY1985.csv


## Contact PIs first name extraction & gender assignment

In [22]:
def string_to_list(string):
    pi_list = []
    for string_part in string.rstrip('; ').split(';'):
        string_part = string_part.strip(', ').replace('"', '')
        if string_part != '':
            pi_list.append(string_part)
    return pi_list


def filter_contact_pi(pi_ids_or_names_list):
    filtered_list = []
    for item in pi_ids_or_names_list: 
         if '(contact)' in item:
            filtered_list.append(item)
    return filtered_list


def extract_contact_pi(pi_ids_or_names_list):
    if len(pi_ids_or_names_list) == 0:
        return None, "no_value"
    elif len(pi_ids_or_names_list) == 1:
        return pi_ids_or_names_list[0], "single_value"
    else:              
        filtered_pi_ids_or_names = filter_contact_pi(pi_ids_or_names_list)
        if len(filtered_pi_ids_or_names) == 0:
            return pi_ids_or_names_list[0], "multiple_values_but_no_explicit_contact_first_chosen"
        elif len(filtered_pi_ids_or_names) == 1:
            return filtered_pi_ids_or_names[0], "multiple_values_and_single_explicit_contact"
        else:
            return filtered_pi_ids_or_names[0], "multiple_values_and_multiple_explicit_contacts_first_chosen"


def normalize_first_name(first_name):
    first_name = re.sub('[\.\-\;\,]', '', first_name)
    first_name = re.sub(' +', ' ', first_name)
    first_name_parts = first_name.strip(' ').split(' ')
    l = []
    for part in first_name_parts:
        if len(part) > 1:
            l.append(part)
    return ' '.join(l).upper()


def extract_pi_first_name(full_name):
    if full_name is None:
        return None
    else:
        full_name_list = full_name.split(',')
        if len(full_name_list) <= 1:
            return None
        else:
            return normalize_first_name(full_name_list[1].replace('(contact)', ''))

In [23]:
# upload and clean gender_US_names.csv list
GENDER_FILE = os.path.join(CURRENT_PATH,'../content/gender_US_names.csv')

with open(GENDER_FILE, 'r', encoding='UTF-8-sig', newline=NEWLINE) as csv_file_descriptor:
    gender_dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')

    gender_dict = {}

    for item in gender_dataset:
        normalized_first_name = normalize_first_name(item['Name'])
        gender_dict[normalized_first_name] = item['Gender']

In [24]:
# enhance output.csv file with contact PIs extraction and gender assignment
with open(ENHANCED_FILE, 'w', encoding=ENCODING, newline=NEWLINE) as output_file:
    writer = csv.DictWriter(output_file, fieldnames=[
        APPLICATION_ID,
        ACTIVITY,
        ADMINISTERING_IC,
        APPLICATION_TYPE,
        ARRA_FUNDED,
        AWARD_NOTICE_DATE,
        BUDGET_START,
        BUDGET_END,
        CFDA_CODE,
        CORE_PROJECT_NUM,
        ED_INST_TYPE,
        FOA_NUMBER,
        FULL_PROJECT_NUM,
        FUNDING_ICS,
        FUNDING_MECHANISM,
        FY,
        IC_NAME,
        NIH_SPENDING_CATS,
        ORG_CITY,
        ORG_COUNTRY,
        ORG_DEPT,
        ORG_DISTRICT,
        ORG_DUNS,
        ORG_FIPS,
        ORG_IPF_CODE,
        ORG_NAME,
        ORG_STATE,
        ORG_ZIPCODE,
        PHR,
        PI_IDS,
        PI_NAMES,

        NB_PI_IDS,
        NB_PI_NAMES,
        CONTACT_PI_ID, 
        CONTACT_PI_NAME, 
        CONTACT_PI_ID_STATUS, 
        CONTACT_PI_NAME_STATUS,
        CONTACT_PI_FIRST_NAME,
        CONTACT_PI_GENDER,

        PROGRAM_OFFICER_NAME,
        PROJECT_START,
        PROJECT_END,
        PROJECT_TERMS,
        PROJECT_TITLE,
        SERIAL_NUMBER,
        STUDY_SECTION,
        STUDY_SECTION_NAME,
        SUBPROJECT_ID,
        SUFFIX,
        SUPPORT_YEAR,
        DIRECT_COST_AMT,
        INDIRECT_COST_AMT,
        TOTAL_COST,
        TOTAL_COST_SUB_PROJECT
    ], delimiter=DELIMITER)
    writer.writeheader()


    with open(OUTPUT_FILE, 'r', encoding=ENCODING, newline=NEWLINE) as input_file:
        reader = csv.DictReader(input_file, delimiter=DELIMITER)

        for index, input_row in enumerate(reader):

            pi_ids_list = string_to_list(input_row[PI_IDS])
            pi_names_list = string_to_list(input_row[PI_NAMES])

            try:
                contact_pi_id, pi_id_status = extract_contact_pi(pi_ids_list)
                contact_pi_name, pi_name_status = extract_contact_pi(pi_names_list)
            except Exception as e:
                print('APPLICATION_ID', input_row[APPLICATION_ID])
                print('PI_IDS', input_row[PI_IDS])
                print('PI_NAMES', input_row[PI_NAMES])
                print('+++')
                print(pi_ids_list)
                print(pi_names_list)
                print(e)
                print('----------------')

            contact_pi_first_name = extract_pi_first_name(contact_pi_name)

            dict_row = {
                APPLICATION_ID: input_row[APPLICATION_ID],
                ACTIVITY: input_row[ACTIVITY],
                ADMINISTERING_IC: input_row[ADMINISTERING_IC],
                APPLICATION_TYPE: input_row[APPLICATION_TYPE],
                ARRA_FUNDED: input_row[ARRA_FUNDED],
                AWARD_NOTICE_DATE: input_row[AWARD_NOTICE_DATE],
                BUDGET_START: input_row[BUDGET_START],
                BUDGET_END: input_row[BUDGET_END],
                CFDA_CODE: input_row[CFDA_CODE],
                CORE_PROJECT_NUM: input_row[CORE_PROJECT_NUM],
                ED_INST_TYPE: input_row[ED_INST_TYPE],
                FOA_NUMBER: input_row[FOA_NUMBER],
                FULL_PROJECT_NUM: input_row[FULL_PROJECT_NUM],
                FUNDING_ICS: input_row[FUNDING_ICS],
                FUNDING_MECHANISM: input_row[FUNDING_MECHANISM],
                FY: input_row[FY],
                IC_NAME: input_row[IC_NAME],
                NIH_SPENDING_CATS: input_row[NIH_SPENDING_CATS],
                ORG_CITY: input_row[ORG_CITY],
                ORG_COUNTRY: input_row[ORG_COUNTRY],
                ORG_DEPT: input_row[ORG_DEPT],
                ORG_DISTRICT: input_row[ORG_DISTRICT],
                ORG_DUNS: input_row[ORG_DUNS],
                ORG_FIPS: input_row[ORG_FIPS],
                ORG_IPF_CODE: input_row[ORG_IPF_CODE],
                ORG_NAME: input_row[ORG_NAME],
                ORG_STATE: input_row[ORG_STATE],
                ORG_ZIPCODE: input_row[ORG_ZIPCODE],
                PHR: input_row[PHR],
                PI_IDS: input_row[PI_IDS],
                PI_NAMES: input_row[PI_NAMES],

                NB_PI_IDS: len(pi_ids_list),
                NB_PI_NAMES: len(pi_names_list),
                CONTACT_PI_ID: contact_pi_id, 
                CONTACT_PI_NAME: contact_pi_name, 
                CONTACT_PI_ID_STATUS: pi_id_status, 
                CONTACT_PI_NAME_STATUS: pi_name_status,
                CONTACT_PI_FIRST_NAME: contact_pi_first_name,
                CONTACT_PI_GENDER: gender_dict.get(contact_pi_first_name),

                PROGRAM_OFFICER_NAME: input_row[PROGRAM_OFFICER_NAME],
                PROJECT_START: input_row[PROJECT_START],
                PROJECT_END: input_row[PROJECT_END],
                PROJECT_TERMS: input_row[PROJECT_TERMS],
                PROJECT_TITLE: input_row[PROJECT_TITLE],
                SERIAL_NUMBER: input_row[SERIAL_NUMBER],
                STUDY_SECTION: input_row[STUDY_SECTION],
                STUDY_SECTION_NAME: input_row[STUDY_SECTION_NAME],
                SUBPROJECT_ID: input_row[SUBPROJECT_ID],
                SUFFIX: input_row[SUFFIX],
                SUPPORT_YEAR: input_row[SUPPORT_YEAR],
                DIRECT_COST_AMT: input_row[DIRECT_COST_AMT],
                INDIRECT_COST_AMT: input_row[INDIRECT_COST_AMT],
                TOTAL_COST: input_row[TOTAL_COST],
                TOTAL_COST_SUB_PROJECT: input_row[TOTAL_COST_SUB_PROJECT]
            }
            writer.writerow(dict_row)

# Designing and building SQL database

Design is being built here: https://dbdiagram.io/d/61fd59c585022f4ee53e2950

Database type: Microsoft SQL Server

Purpose: usage by the CRC team members

# Table creation

In [25]:
def filter_columns(source_file, target_file, subset_headers):
    
    subset_headers_set = set(subset_headers)
    
    with open(source_file, 'r', encoding=ENCODING, newline=NEWLINE) as csv_file_descriptor:
        reader = csv.DictReader(csv_file_descriptor, delimiter=DELIMITER)

        with open(target_file, 'w', encoding=ENCODING, newline=NEWLINE) as output_file:
            writer = csv.DictWriter(output_file, fieldnames=subset_headers, delimiter=DELIMITER)
            writer.writeheader()

            for row in reader:
                subset_row = {}
                for key, value in row.items():
                    if key in subset_headers_set:
                        subset_row[key] = value
                writer.writerow(subset_row)

In [26]:
# TABLE PROJECTS
filter_columns(ENHANCED_FILE, TABLE_DIR + 'PROJECTS.tsv', [
    APPLICATION_ID, 
    CORE_PROJECT_NUM, 
    ACTIVITY, 
    ADMINISTERING_IC, 
    APPLICATION_TYPE, 
    ARRA_FUNDED, 
    AWARD_NOTICE_DATE, 
    CFDA_CODE, 
    FOA_NUMBER, 
    FUNDING_ICS, 
    FUNDING_MECHANISM, 
    NIH_SPENDING_CATS, 
    ORG_IPF_CODE, 
    PHR, 
    PROGRAM_OFFICER_NAME, 
    PROJECT_START, 
    PROJECT_END, 
    PROJECT_TERMS, 
    PROJECT_TITLE, 
    SERIAL_NUMBER, 
    SUFFIX, 
    STUDY_SECTION])

In [27]:
# TABLE SUBPROJECTS
filter_columns(ENHANCED_FILE, TABLE_DIR + 'SUBPROJECTS.tsv', [
    SUBPROJECT_ID, 
    PROJECT_TITLE, 
    TOTAL_COST_SUB_PROJECT])

In [28]:
# TABLE ACTIVITY
filter_columns(ENHANCED_FILE, TABLE_DIR + 'ACTIVITY.tsv', [
    ACTIVITY, 
    'ACTIVITY_NAME']) #champ vide pour l'instant; à compléter avec documentation du NIH

In [29]:
# TABLE APPLICATION_TYPE
filter_columns(ENHANCED_FILE, TABLE_DIR + 'APPLICATION_TYPE.tsv', [
    APPLICATION_TYPE, 
    'APPLICATION_TYPE_NAME']) #champ vide pour l'instant; à compléter avec documentation du NIH

In [30]:
# TABLE PRINCIPAL_INVESTIGATORS
filter_columns(ENHANCED_FILE, TABLE_DIR + 'PRINCIPAL_INVESTIGATORS.tsv', [
    APPLICATION_ID, 
    PI_IDS, 
    PI_NAMES, 
    NB_PI_IDS, 
    NB_PI_NAMES, 
    CONTACT_PI_ID, 
    CONTACT_PI_NAME, 
    CONTACT_PI_FIRST_NAME, 
    CONTACT_PI_GENDER])

In [31]:
# TABLE FUNDING_PER_FISCAL_YEAR
filter_columns(ENHANCED_FILE, TABLE_DIR + 'FUNDING_PER_FISCAL_YEAR.tsv', [
    APPLICATION_ID, 
    FULL_PROJECT_NUM, 
    FY, 
    BUDGET_START, 
    BUDGET_END, 
    SUPPORT_YEAR, 
    DIRECT_COST_AMT, 
    INDIRECT_COST_AMT, 
    TOTAL_COST])

In [32]:
# TABLE GRANTEE_ORGANIZATION
filter_columns(ENHANCED_FILE, TABLE_DIR + 'GRANTEE_ORGANIZATION.tsv', [
    ORG_IPF_CODE, 
    ORG_DUNS, 
    ORG_NAME, 
    ORG_DEPT, 
    ORG_FIPS, 
    ORG_COUNTRY, 
    ORG_STATE, 
    ORG_DISTRICT, 
    ORG_CITY, 
    ORG_ZIPCODE, 
    ED_INST_TYPE])

In [33]:
# TABLE INSTITUTE_OR_CENTER
filter_columns(ENHANCED_FILE, TABLE_DIR + 'INSTITUTE_OR_CENTER.tsv', [
    ADMINISTERING_IC, 
    IC_NAME, 
    'IC_ACRONYM']) #champ vide pour l'instant; à compléter avec documentation du NIH

In [34]:
# TABLE STUDY_SECTION
filter_columns(ENHANCED_FILE, TABLE_DIR + 'STUDY_SECTION.tsv', [
    STUDY_SECTION, 
    STUDY_SECTION_NAME])

## QA

In [ ]:
df = pd.read_csv(TABLE_DIR + 'PROJECTS.tsv')

In [ ]:
df.head()

## Approche testée mais rejetée

On conservera plutôt les titres des champs du NIH même s'ils manquent parfois de clarté, pour faciliter la mise à jour et reprise du projet par d'autres.

def rename_headers(source_file, target_file, renamed_header_dict):
    
    with open(source_file, 'r', encoding='utf-8') as csv_file_descriptor:
        reader = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')

        with open(target_file, 'w', encoding='utf-8') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=renamed_header_dict.values(), dialect=csv.excel)
            writer.writeheader()

            for row in reader:
                new_row = {}
                for old_header, value in row.items():
                    new_header = renamed_header_dict[old_header] #syntaxe pour dire: prends la valeur sous la clé 'old_header'
                    new_row[new_header] = value #ajouter nouvelle clé, valeur au dictionnaire new_row
                writer.writerow(new_row)
    

def filter_and_rename(input_file, output_file, columns):    
    #appeler les deux autres fonctions
    temp_file_name = f'temp/{input_file}'
    
filter_and_rename('content/enhanced-output.csv', 'content/table-projects.csv', OrderedDict([
    (APPLICATION_ID, APPLICATION_ID),
    (CORE_PROJECT_NUM, CORE_PROJECT_NUM),
    (ACTIVITY, 'ACTIVITY_CODE'),
    (ADMINISTERING_IC, 'ADMINISTERING_IC_CODE'),
    (APPLICATION_TYPE, 'APPLICATION_TYPE_CODE'),
    (ARRA_FUNDED, ARRA_FUNDED)]))

TO DO

tester ce que le renamed_header_dict.values() fait, en créant un dictionnaire vide

grosso modo ça crée une liste avec les valeurs du renamed_header_dict (qui va devenir notre orderedDict quand on va appeler la fonction plus loin)
donc on va chercher le nouveau header qui est la valeur (on aurait fait renamed_header_dict.keys() si on avait voulu la clé, i.e. anciens headers)
optionnellement rajouter avec un .lower()

ensuite, continuer avec la fonction filter_and_rename
puis compléter le orderedDict avec tous les headers pour la table Projects